In [1]:
import pandas as pd
import numpy as np

In [2]:
from constants import cleaned_data_filepath
participants = ["107", "904", "905", "906"]

from ml_prep import load_ml_data
df, ys = load_ml_data(cleaned_data_filepath, participants)

In [3]:
from extracted_features.ml_prep_tsfresh import prep_tsfresh_data
import extracted_features.ml_prep_tsfresh as m_prep
import importlib
importlib.reload(m_prep)
df, extracted = prep_tsfresh_data(df, ys, participants)

Feature Extraction: 100%|█████████████████████████████████████████████████████████████████████████████████████| 10/10 [59:54<00:00, 359.40s/it]


In [4]:
extracted.to_csv(path_or_buf="extracted.csv")

In [5]:
extracted

variable,"E32__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8","E52__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2","E32__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.8","E25__fft_coefficient__coeff_8__attr_""abs""","E32__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.8","E52__fft_coefficient__coeff_9__attr_""abs""","E52__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2","E32__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4","E32__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.4","E52__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2",...,"E7__change_quantiles__f_agg_""var""__isabs_True__qh_0.2__ql_0.0","E19__fft_coefficient__coeff_5__attr_""abs""","E13__fft_coefficient__coeff_17__attr_""abs""","E13__fft_coefficient__coeff_18__attr_""abs""","E7__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.2",E23__spkt_welch_density__coeff_2,"E28__fft_coefficient__coeff_4__attr_""abs""",E30__ratio_beyond_r_sigma__r_3,E60__abs_energy,"E13__fft_coefficient__coeff_19__attr_""abs"""
id,,,,,,,,,,,,,,,,,,,,,
0,0.733647,0.744670,0.810957,90.002808,0.292480,189.823037,0.882424,0.797009,0.838379,0.708676,...,0.460352,787.757539,28.070167,25.720784,0.334672,5.646973e+03,576.466300,0.000,3.614092e+05,23.591600
1,0.775345,0.910937,1.050360,224.156385,0.460448,273.301982,1.401638,1.118326,2.069926,0.754035,...,0.130763,32.340672,13.285350,12.596274,0.164080,3.018835e+03,329.254713,0.000,7.884323e+03,11.951404
2,0.345309,1.106713,0.246504,19.571514,0.127650,289.383094,1.780595,0.762074,1.035701,1.182054,...,0.290700,260.852246,41.485937,38.790485,0.274974,4.086199e+03,263.594550,0.000,2.511400e+05,36.464479
3,0.861328,0.802935,0.787071,163.315617,0.486984,106.833873,0.868170,0.941376,1.151605,0.899559,...,0.057944,266.817704,61.686046,56.670153,0.063575,2.540979e+03,184.925999,0.000,3.288350e+04,52.493809
4,0.970663,0.872971,1.178229,75.674607,0.236107,213.324559,1.101571,1.333204,2.537468,0.910265,...,0.020847,416.149863,10.070639,9.205055,0.234808,2.664495e+03,329.629319,0.000,2.103392e+04,8.467799
5,0.742810,0.820151,0.666281,206.627356,0.175069,179.874842,1.073691,0.862606,0.925823,0.887181,...,0.096273,423.894236,14.453567,13.801686,0.077705,4.342541e+03,193.826704,0.000,6.700013e+04,13.189503
6,0.332702,0.709517,0.167941,111.182463,0.096020,221.873772,0.725250,0.654234,0.641252,0.774014,...,0.089490,392.699318,54.660943,51.942108,0.195009,6.799351e+02,217.206065,0.000,7.276327e+04,49.487789
7,0.806030,0.850474,0.967029,311.326928,0.317612,249.618460,1.295894,0.889339,1.122459,0.584902,...,0.069073,324.943660,36.852526,33.891630,0.089189,3.773450e+02,346.348469,0.000,6.710883e+04,31.412989
8,1.833564,0.914098,5.040224,198.371705,1.686644,218.103194,1.307007,2.025214,6.743955,0.945746,...,0.198355,662.327806,143.760979,135.906239,0.224238,2.884325e+03,224.691151,0.000,3.975224e+04,128.915266


In [7]:
from bad_trials import get_bad_trials, transform_ybad_indices
from first_participants_map import map_first_participants

ybad = get_bad_trials(participants, ys)
ys = map_first_participants(ys, participants)
y = np.concatenate(ys)
ybad = transform_ybad_indices(ybad, ys)
y[ybad] = -1

In [9]:
y.shape

(374,)

In [14]:
extracted.shape

(374, 5395)

In [16]:
extracted["label"]=y

In [19]:
edf = extracted[extracted.label != -1]

In [20]:
edf.shape

(219, 5396)

In [21]:
edf.label -= 1

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [22]:
edf.label

id
0       8
3       2
4      11
6       9
7       6
       ..
354     0
355     8
356     6
359     9
368     6
Name: label, Length: 219, dtype: int32

In [23]:
edf.label.max()

14

In [24]:
edf.label.min()

0

In [28]:
edf[edf.label < 8].label = 0
edf[edf.label >= 8].label = 1

In [29]:
edf.label

id
0      1
3      0
4      1
6      1
7      0
      ..
354    0
355    1
356    0
359    1
368    0
Name: label, Length: 219, dtype: int32

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
model = LinearSVC()
from sklearn.model_selection import GridSearchCV

param_grid = [
 {'C': np.logspace(-10, 10, 10)}
]

tuned_model = GridSearchCV(model, param_grid)
tuned_model.fit(edf, y)

tuned_model.cv_results_

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\karl\appdata\l

{'mean_fit_time': array([2.56420946, 2.41980577, 2.29934708, 2.2667009 , 2.31982827,
        2.3935554 , 2.29312412, 2.28484845, 2.32711498, 2.49496913]),
 'std_fit_time': array([0.20443735, 0.01387562, 0.09399463, 0.10253448, 0.17963465,
        0.24767903, 0.0471314 , 0.1161329 , 0.07055552, 0.38837518]),
 'mean_score_time': array([0.00332467, 0.00300717, 0.00300233, 0.00317383, 0.00299946,
        0.00364621, 0.00266854, 0.00333293, 0.00271273, 0.00367363]),
 'std_score_time': array([4.59968055e-04, 6.66151352e-06, 7.19042749e-06, 2.52039075e-04,
        8.16539379e-04, 9.79127570e-04, 4.71314650e-04, 4.69630553e-04,
        5.08475269e-04, 9.52463421e-04]),
 'param_C': masked_array(data=[1e-10, 1.6681005372000592e-08, 2.782559402207126e-06,
                    0.0004641588833612782, 0.07742636826811278,
                    12.915496650148826, 2154.4346900318865,
                    359381.36638046405, 59948425.03189421, 10000000000.0],
              mask=[False, False, False, False

In [55]:
tuned_model.best_estimator_

LinearSVC(C=10000000000.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [31]:
y = np.array(edf.label)

In [32]:
edf = edf.drop(columns=["label"], axis=1)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(edf, y, test_size=0.25)

In [50]:
model.fit(X_train, y_train)

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
xhat = model.predict(X_test)

In [52]:
np.mean(xhat != y_test)

0.509090909090909